In [1]:
import numpy as np
import pandas as pd
import random
import math
import matplotlib as plt
import itertools

In [117]:
def mmn_queueing_jsq(n,arrival_rate,service_rate,simulation_time=60,simuseed=math.nan):
    '''
    Simulating the mmn queueing system using the method of JSQ 
    (namely join the shortest queue method), the user needs to provide
    the number of servers (n),arrival rate (lambda) and service rate (mu),
    in case of n >= 2, if mu is provided as a scaler, then all servers
    share the same mu, if mu is provided as a vector, then all servers may have different mu,
    the simulation time is by default 60 units, and for reproducibility purposes, the user
    can define simulation seed, by default the seed value is not set.
    
    n:               the number of servers, a positive finite integer
    arrival_rate:    exponential rate of arrival
    service_rate:    exponential rate of service time
    simulation_time: the simulation time, a positive finite number
    simuseed:        the simulation seed number
    
    
    There are 7 outputs in this function,
    event_calendar:   the event calendar for the whole simulation, containing all important
                      information needed (current time, next customer arrival time, finish time,
                      server occupancy variable, server next available time, server queue length,
                      server total, grand queue length and grand total)
    utiliz:           the utilization for each server
    ans:              the average number in the system for each server and grand average
    aql:              the average queue length for each server and grand average
    awt:              the average waiting time for each customer and grand average
    act:              the average cycle time for each customer and grand average
    no_cust_arr_comp: the table illustrating the number of arrived customers and the
                      the number of completed services for each server and grand total
    
    Code writer: Yaozu Li
    
    '''
    
    if math.isnan(simuseed) == True:
        pass
    else:
        random.seed(simuseed)

    if not type(n) is int:
        raise TypeError("n needs to be integer")
    if n < 0:
        raise Exception("n needs to be positive")
    
    lamb =  arrival_rate
    
    if isinstance(service_rate, list):
        if len(service_rate) == n:
            mu = service_rate
        elif (n > 1 and len(service_rate) == 1):
            mu = n*service_rate
        else:
            raise Exception("Unmatched cardinality for service rate and n")
    elif isinstance(service_rate, int):
        if n == 1:
            mu = [service_rate]
        elif n > 1:
            mu = n*[service_rate]
    else:
        raise Exception("Please use int or list type for service rate")
    
    
    # Simulating the event calendar
    table_col = np.arange(n)*4+3
    avail_col = np.arange(n)*4+4
    queue_col = np.arange(n)*4+5
    total_col = np.arange(n)*4+6
    compare_col = np.insert(avail_col,0,[1,2])
    
    event_calendar = np.zeros([2,3+4*n])
    event_calendar[-1,1] = random.expovariate(lamb)
    event_calendar[-1,2] = simulation_time
    event_calendar[-1,avail_col] = math.nan
    
    no_customers = np.zeros(n+1)  # keep track of the number customers
    no_completed_service = np.zeros(n+1)   # keep track of the number of completed services
    
    while event_calendar[-1,0] < simulation_time:
        event_calendar = np.vstack([event_calendar,event_calendar[-1,:]])
        event_calendar[-1,0] = min(event_calendar[-2,compare_col])
        
        # If the next event is an arrival
        if event_calendar[-1,0] == event_calendar[-2,1]:
            event_calendar[-1,1] = event_calendar[-1,0]+random.expovariate(lamb)
            server_assignment = random.choice(np.where(event_calendar[-1,total_col] == \
                                   min(event_calendar[-1,total_col]))[0])
            no_customers[server_assignment] += 1
            
            # If the server is not occupied
            if event_calendar[-1,table_col[server_assignment]] == 0:
                event_calendar[-1,table_col[server_assignment]] += 1
                event_calendar[-1,avail_col[server_assignment]] = event_calendar[-1,0] + \
                                                        random.expovariate(mu[server_assignment])
                event_calendar[-1,total_col[server_assignment]] += 1
            
            # Else if the server is occupied
            elif event_calendar[-1,table_col[server_assignment]] == 1:
                event_calendar[-1,queue_col[server_assignment]] += 1
                event_calendar[-1,total_col[server_assignment]] += 1
            
        # Else if the next event is the simulation termination
        elif event_calendar[-1,0] == simulation_time:
            pass
        
        # Else if the next event is a service completion
        else:
            server_completion = np.where(event_calendar[-2,avail_col] == event_calendar[-1,0])[0][0]
            no_completed_service[server_completion] += 1
            
            # If there is no queue behind
            if event_calendar[-1,queue_col[server_completion]] == 0:
                event_calendar[-1,table_col[server_completion]] -= 1
                event_calendar[-1,avail_col[server_completion]] = math.nan
                event_calendar[-1,total_col[server_completion]] -= 1
            
            # Else if there is a queue behind
            elif event_calendar[-1,queue_col[server_completion]] > 0:
                event_calendar[-1,avail_col[server_completion]] = event_calendar[-1,0] + \
                                                        random.expovariate(mu[server_completion])
                event_calendar[-1,queue_col[server_completion]] -= 1
                event_calendar[-1,total_col[server_completion]] -= 1
    
    no_customers[-1] = sum(no_customers[:-1])
    no_completed_service[-1] = sum(no_completed_service[:-1])
    event_calendar = pd.DataFrame(event_calendar).tail(-1)
    event_calendar.columns = ['Time','Next Customer','Finish Time'] + \
                    list(itertools.chain.from_iterable([[f"Server{i}", \
                    f"Server{i} Available Time",f"Server{i} Queue",f"Server{i} Total"] for i in range(1,n+1)]))
    event_calendar["Grand Queue"] = event_calendar.iloc[:,queue_col].sum(axis=1)
    event_calendar["Grand Total"] = event_calendar.iloc[:,total_col].sum(axis=1)
    
    # Concluding performance measures
    # Time dynamic
    time1 = event_calendar.iloc[:-1,0].to_numpy()
    time2 = event_calendar.iloc[1:,0].to_numpy()
    time3 = time2 - time1
    
    # Calculate the utilization for each server
    utiliz_server = event_calendar.iloc[:-1,table_col].to_numpy()
    utiliz_data = ((time3 @ utiliz_server)/simulation_time).round(3)
    utiliz = pd.DataFrame(utiliz_data).transpose()
    utiliz.columns = [f"Server{i} Utilization" for i in range(1,n+1)]
    
    # Calculating the average number in the systems for each server and grand total
    ans_server = event_calendar.iloc[:-1,np.append(total_col,event_calendar.shape[1]-1)].to_numpy()
    ans_data = ((time3 @ ans_server)/simulation_time).round(3)
    ans = pd.DataFrame(ans_data).transpose()
    ans.columns = [f"Server{i} Average number in the system" for i in range(1,n+1)]+["Grand Average number in the system"]
    
    # Calculating the average queue length
    aql_server = event_calendar.iloc[:-1,np.append(queue_col,event_calendar.shape[1]-2)].to_numpy()
    aql_data = ((time3 @ aql_server)/simulation_time).round(3)
    aql = pd.DataFrame(aql_data).transpose()
    aql.columns = [f"Server{i} Average queue length" for i in range(1,n+1)]+["Grand Average queue length"]
    
    # Average waiting time
    awt_data = ((time3 @ aql_server)/no_customers).round(3)
    awt = pd.DataFrame(awt_data).transpose()
    awt.columns = [f"Server{i} Average waiting time" for i in range(1,n+1)]+["Grand Average waiting time"]
    
    # Average cycle time
    act_data = ((time3 @ ans_server)/no_customers).round(3)
    act = pd.DataFrame(act_data).transpose()
    act.columns = [f"Server{i} Average cycle time" for i in range(1,n+1)]+["Grand Average cycle time"]
    
    # Show the number of customers arrived and completed
    no_cust_arr_comp = pd.DataFrame(np.vstack([no_customers,no_completed_service]))
    no_cust_arr_comp.columns = [f"Server{i}" for i in range(1,n+1)]+["Grand total"]
    no_cust_arr_comp.index = ["Arrived customers", "Completed services"]
    
    # Return the event calendar and all performance measures table
    return event_calendar, utiliz, ans, aql, awt, act, no_cust_arr_comp


In [ ]:
def queue_length_time_graph():
    # Ploting the queue length time graph
    
    
    
    
    

In [108]:
mmn_queueing_jsq(4,20,[3,8,10,5],simulation_time=100,simuseed=8)[0]

,Time,Next Customer,Finish Time,Server1,Server1 Available Time,Server1 Queue,Server1 Total,Server2,Server2 Available Time,Server2 Queue,...,Server3,Server3 Available Time,Server3 Queue,Server3 Total,Server4,Server4 Available Time,Server4 Queue,Server4 Total,Grand Queue,Grand Total
1,0.000000,0.012855,100.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,...,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0
2,0.012855,0.176753,100.0,0.0,NaN,0.0,0.0,1.0,0.039677,0.0,...,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0
3,0.039677,0.176753,100.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,...,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0
4,0.176753,0.178991,100.0,0.0,NaN,0.0,0.0,1.0,0.212287,0.0,...,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0
5,0.178991,0.531258,100.0,1.0,0.349661,0.0,1.0,1.0,0.212287,0.0,...,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3858,99.900589,99.918337,100.0,1.0,99.968331,1.0,2.0,0.0,NaN,0.0,...,0.0,NaN,0.0,0.0,1.0,100.02868,0.0,1.0,1.0,3.0
3859,99.918337,99.979465,100.0,1.0,99.968331,1.0,2.0,0.0,NaN,0.0,...,1.0,100.221605,0.0,1.0,1.0,100.02868,0.0,1.0,1.0,4.0
3860,99.968331,99.979465,100.0,1.0,100.848260,0.0,1.0,0.0,NaN,0.0,...,1.0,100.221605,0.0,1.0,1.0,100.02868,0.0,1.0,0.0,3.0
3861,99.979465,100.003006,100.0,1.0,100.848260,0.0,1.0,1.0,100.109495,0.0,...,1.0,100.221605,0.0,1.0,1.0,100.02868,0.0,1.0,0.0,4.0


In [109]:
mmn_queueing_jsq(4,20,[3,8,10,5],simulation_time=100,simuseed=8)[1]

,Server1 Utilization,Server2 Utilization,Server3 Utilization,Server4 Utilization
0,0.853,0.719,0.661,0.809


In [116]:
mmn_queueing_jsq(4,20,[3,8,10,5],simulation_time=100,simuseed=8)[2]

,Server1 Average number in the system,Server2 Average number in the system,Server3 Average number in the system,Server4 Average number in the system,Grand Average number in the system
0,1.569,1.157,1.075,1.385,5.187


In [111]:
mmn_queueing_jsq(4,20,[3,8,10,5],simulation_time=100,simuseed=8)[3]

,Server1 Average queue length,Server2 Average queue length,Server3 Average queue length,Server4 Average queue length,Grand Average queue length
0,0.716,0.439,0.414,0.576,2.145


In [112]:
mmn_queueing_jsq(4,20,[3,8,10,5],simulation_time=100,simuseed=8)[4]

,Server1 Average waiting time,Server2 Average waiting time,Server3 Average waiting time,Server4 Average waiting time,Grand Average waiting time
0,0.246,0.076,0.064,0.138,0.111


In [113]:
mmn_queueing_jsq(4,20,[3,8,10,5],simulation_time=100,simuseed=8)[5]

,Server1 Average cycle time,Server2 Average cycle time,Server3 Average cycle time,Server4 Average cycle time,Grand Average cycle time
0,0.539,0.2,0.167,0.331,0.268


In [114]:
mmn_queueing_jsq(4,20,[3,8,10,5],simulation_time=100,simuseed=8)[6]

,Server1,Server2,Server3,Server4,Grand total
Arrived customers,291.0,579.0,644.0,418.0,1932.0
Completed services,290.0,578.0,643.0,417.0,1928.0
